<a href="https://colab.research.google.com/github/iCorv/raw_audio_autoencoder/blob/master/raw_audio_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib
import tensorflow as tf
from tensorflow.keras.layers import Convolution2D, UpSampling2D, AveragePooling2D, Convolution2DTranspose, Input
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal



Using matplotlib backend: agg


In [31]:
AUDIO_CHUNK_SIZE = 1024
RATE = 44100

# build the model
kernel_size = (1,7)
input_audio = Input(shape=(1, AUDIO_CHUNK_SIZE, 1))

net = Convolution2D(filters=64, kernel_size=kernel_size,activation='relu',padding='same')(input_audio)
net = AveragePooling2D((1,2))(net)
net = Convolution2D(filters=32, kernel_size=kernel_size,activation='relu',padding='same')(net)
net = AveragePooling2D((1,2))(net)
net = Convolution2D(filters=4, kernel_size=kernel_size, activation='relu',padding='same')(net)
net = AveragePooling2D((1,2))(net)
net = Convolution2D(filters=2, kernel_size=kernel_size, activation='relu',padding='same')(net)

# audio chunk encoded in 128 data points
encoded = AveragePooling2D((1,2), name='encoded_audio')(net)

net = Convolution2DTranspose(filters=2, kernel_size=kernel_size, activation='relu', padding='same')(encoded)
net = UpSampling2D((1,2))(net)
net = Convolution2DTranspose(filters=4, kernel_size=kernel_size, activation='relu', padding='same')(net)
net = UpSampling2D((1,2))(net)
net = Convolution2DTranspose(filters=32, kernel_size=kernel_size, activation='relu', padding='same')(net)
net = UpSampling2D((1,2))(net)
net = Convolution2DTranspose(filters=64, kernel_size=kernel_size, activation='relu', padding='same')(net)
net = UpSampling2D((1,2))(net)

decoded = Convolution2DTranspose(filters=1, kernel_size=kernel_size, activation='linear', padding='same', name='decoded_audio')(net)

autoencoder = Model(input_audio, decoded, name='simple_autoencoder')
autoencoder.summary()

Model: "simple_autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1, 1024, 1)]      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 1, 1024, 64)       512       
_________________________________________________________________
average_pooling2d_9 (Average (None, 1, 512, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 1, 512, 32)        14368     
_________________________________________________________________
average_pooling2d_10 (Averag (None, 1, 256, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 256, 4)         900       
_________________________________________________________________
average_pooling2d_11 (Averag (None, 1, 128, 4)  

In [0]:
# lets build an audio dataset of random sinus chunks!

def sinus_dataset_generator(num_examples, fs, samples, frequency_range):
    """Builds a dataset of sinus.
    
    Args:
        num_examples: number of examples to generate (int)
        fs: sample rate of the sinus
        samples: number of samples to generate (int)
        frequency_range: a list of two values defining [lower, upper] frequency range (int)
    Returns:
        A numpy array of sinus examples.
    """
    # first example
    sinus_data = (np.sin((2*np.pi*np.arange(samples)*440.0/fs) + 0.0)).astype(np.float32)
    sinus_data = np.reshape(sinus_data, newshape=(1, 1, samples, 1))
    
    for idx in range(0, num_examples-1): 
        # random frequency
        f = np.random.randint(frequency_range[0], frequency_range[1])
        # random phase shift
        phase = np.random.random() * np.pi
        # random gain
        gain = np.random.uniform(0.5, 1.0)

        sinus = (np.sin((2*np.pi*np.arange(samples)*f/fs) + phase) * gain).astype(np.float32)
        # add some noise, mu = 0, sigma = 0.1
        s = np.random.normal(0, 0.1, samples)
        sinus = sinus + s
        # bring it into shape for the model 
        sinus = np.reshape(sinus, newshape=(1, 1, samples, 1))
        sinus_data = np.append(sinus_data, sinus, axis=0)
    return sinus_data

num_examples = 8000
sinus_data = sinus_dataset_generator(num_examples, RATE, AUDIO_CHUNK_SIZE, [30, 8000])

# split into train and eval dataset, roughly a 70/30 split
split = int(num_examples * 0.7)
train_dataset = tf.data.Dataset.from_tensor_slices((sinus_data[:split], sinus_data[:split]))
eval_dataset = tf.data.Dataset.from_tensor_slices((sinus_data[split:], sinus_data[split:]))

# see if its actually a sinus...
fig, ax = plt.subplots(3, 3)
for ii in range(3):
  for jj in range(3):
    ax[ii, jj].plot(np.squeeze(sinus_data[np.random.randint(0, num_examples-1)])[:200], color="tab:blue")
    ax[ii, jj].set_xticks([])
    ax[ii, jj].set_yticks([])
fig.savefig('/tmp/sinus_examples.pdf')

In [34]:
# compile the model with adam optimizer and mean squeared error as its loss function
autoencoder.compile(optimizer='adam', loss='mse', metrics=['mse'])

# some constants for training
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

# shuffle and batch the examples
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = eval_dataset.batch(BATCH_SIZE)

# train the model
autoencoder.fit(train_dataset, epochs=100, validation_data=eval_dataset)

Train on 175 steps, validate on 75 steps
Epoch 1/100
175/175 [==============================] - 2s 11ms/step - loss: 0.2586 - mean_squared_error: 0.2586 - val_loss: 0.2437 - val_mean_squared_error: 0.2437
Epoch 2/100
175/175 [==============================] - 1s 8ms/step - loss: 0.2189 - mean_squared_error: 0.2189 - val_loss: 0.1954 - val_mean_squared_error: 0.1954
Epoch 3/100
175/175 [==============================] - 1s 7ms/step - loss: 0.1741 - mean_squared_error: 0.1741 - val_loss: 0.1669 - val_mean_squared_error: 0.1669
Epoch 4/100
175/175 [==============================] - 1s 7ms/step - loss: 0.1499 - mean_squared_error: 0.1499 - val_loss: 0.1289 - val_mean_squared_error: 0.1289
Epoch 5/100
175/175 [==============================] - 1s 7ms/step - loss: 0.0964 - mean_squared_error: 0.0964 - val_loss: 0.0852 - val_mean_squared_error: 0.0852
Epoch 6/100
175/175 [==============================] - 1s 8ms/step - loss: 0.0767 - mean_squared_error: 0.0767 - val_loss: 0.0731 - val_mean_sq

In [0]:
# plot some examples of sinus and its reconstruction
fig, ax = plt.subplots(3, 3)
for ii in range(3):
  for jj in range(3):
    sinus = sinus_data[ii * 3 + jj + 1]
    sinus = np.expand_dims(sinus, axis=0)
    prediction = autoencoder.predict(sinus, batch_size=1)
    ax[ii, jj].plot(np.squeeze(sinus)[:200], color="tab:blue")
    ax[ii, jj].plot(np.squeeze(prediction)[:200], color="tab:red")
    ax[ii, jj].set_xticks([])
    ax[ii, jj].set_yticks([])
fig.savefig('/tmp/sinus_reconstruction.pdf')

In [0]:
# so what happens if we feed audio into the model which is not a sine wave...

# ...but a sawtooth?
t = np.linspace(0, AUDIO_CHUNK_SIZE, AUDIO_CHUNK_SIZE)
sawtooth = signal.sawtooth(2 * np.pi * 100 * t)

# plot original signal
fig = plt.figure()
plt.plot(t[:200], sawtooth[:200], color="tab:blue")

sawtooth = np.reshape(sawtooth, newshape=(1, 1, AUDIO_CHUNK_SIZE, 1))

# encode and decode the sawtooth
prediction = autoencoder.predict(sawtooth, batch_size=1)

# plot reconstructed signal
plt.plot(t[:200], np.squeeze(prediction)[:200], color="tab:red")
fig.savefig('/tmp/sawtooth_reconstruction.pdf')

In [37]:
# what does the mse tell us?

mse = ((np.squeeze(prediction) - np.squeeze(sawtooth))**2).mean(axis=0)

print("Mean squared error for reconstructing the sawtooth signal: {:.2f}".format(mse))


Mean squared error for reconstructing the sawtooth signal: 0.14
